<a href="https://colab.research.google.com/github/BhoomiShakya/Optuna/blob/main/optuna_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 5.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-11-25 19:52:41,071] A new study created in memory with name: no-name-805d1125-6aeb-4976-b339-94565ef24f8e
[I 2025-11-25 19:52:41,971] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 112, 'max_depth': 13}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-25 19:52:42,811] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 116, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-25 19:52:45,134] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 182, 'max_depth': 10}. Best is trial 2 with value: 0.7709497206703911.
[I 2025-11-25 19:52:47,372] Trial 3 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 164, 'max_depth': 18}. Best is trial 2 with value: 0.7709497206703911.
[I 2025-11-25 19:52:49,851] Trial 4 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 115, 'max_depth': 6}. Best is trial 2 with value: 0.770949

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 126, 'max_depth': 16}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-11-25 19:53:34,063] A new study created in memory with name: no-name-5f475fc4-7c1a-45df-a51e-0bb2d5bd9f23
[I 2025-11-25 19:53:35,122] Trial 0 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 191, 'max_depth': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-11-25 19:53:35,817] Trial 1 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 119, 'max_depth': 11}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-11-25 19:53:36,820] Trial 2 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 183, 'max_depth': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-11-25 19:53:37,615] Trial 3 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 145, 'max_depth': 5}. Best is trial 0 with value: 0.7672253258845437.
[I 2025-11-25 19:53:38,732] Trial 4 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 188, 'max_depth': 19}. Best is trial 4 with value: 0.7765363

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 123, 'max_depth': 18}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-11-25 19:54:17,544] A new study created in memory with name: no-name-aabf0af8-9009-4b36-af6c-6189587ba64c
[I 2025-11-25 19:54:18,121] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-25 19:54:19,332] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-11-25 19:54:19,800] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-11-25 19:54:20,753] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-11-25 19:54:21,500] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
!pip install plotly


In [18]:
!pip install nbformat


In [19]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [20]:
# 1. Optimization History
plot_optimization_history(study).show()

In [21]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [22]:
# 3. Slice Plot
plot_slice(study).show()

In [23]:
# 4. Contour Plot
plot_contour(study).show()

In [24]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [25]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [26]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [27]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-11-25 19:54:51,295] A new study created in memory with name: no-name-82459121-5f6d-4d7d-830a-d654e83a9d24
[I 2025-11-25 19:54:54,889] Trial 0 finished with value: 0.7355679702048418 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 120, 'learning_rate': 0.03195532142696626, 'max_depth': 16, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.7355679702048418.
[I 2025-11-25 19:54:57,281] Trial 1 finished with value: 0.7523277467411544 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 76, 'learning_rate': 0.0433551494297765, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 4}. Best is trial 1 with value: 0.7523277467411544.
[I 2025-11-25 19:54:57,355] Trial 2 finished with value: 0.7839851024208566 and parameters: {'classifier': 'SVM', 'C': 0.33864862703078796, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 2 with value: 0.7839851024208566.
[I 2025-11-25 19:55:04,425] Trial 3 finished with value: 0.750465

In [28]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12953343004551354, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [29]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.735568,2025-11-25 19:54:51.297316,2025-11-25 19:54:54.888932,0 days 00:00:03.591616,NaN,NaN,GradientBoosting,NaN,NaN,0.031955,16.0,4.0,5.0,120.0,COMPLETE
1,1,0.752328,2025-11-25 19:54:54.890227,2025-11-25 19:54:57.281797,0 days 00:00:02.391570,NaN,NaN,GradientBoosting,NaN,NaN,0.043355,8.0,4.0,7.0,76.0,COMPLETE
2,2,0.783985,2025-11-25 19:54:57.285656,2025-11-25 19:54:57.355239,0 days 00:00:00.069583,0.338649,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.750466,2025-11-25 19:54:57.356091,2025-11-25 19:55:04.425059,0 days 00:00:07.068968,NaN,NaN,GradientBoosting,NaN,NaN,0.074373,8.0,9.0,6.0,233.0,COMPLETE
4,4,0.761639,2025-11-25 19:55:04.427329,2025-11-25 19:55:07.870945,0 days 00:00:03.443616,NaN,True,RandomForest,NaN,NaN,NaN,11.0,8.0,2.0,176.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.785847,2025-11-25 19:55:43.682686,2025-11-25 19:55:43.714297,0 days 00:00:00.031611,0.212823,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.759777,2025-11-25 19:55:43.715226,2025-11-25 19:55:44.511247,0 days 00:00:00.796021,NaN,True,RandomForest,NaN,NaN,NaN,18.0,9.0,7.0,137.0,COMPLETE
97,97,0.787709,2025-11-25 19:55:44.512236,2025-11-25 19:55:44.542727,0 days 00:00:00.030491,0.100265,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.746741,2025-11-25 19:55:44.543614,2025-11-25 19:55:47.002964,0 days 00:00:02.459350,NaN,NaN,GradientBoosting,NaN,NaN,0.060655,10.0,7.0,5.0,198.0,COMPLETE


In [30]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
GradientBoosting,11
RandomForest,10


In [31]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.744371
RandomForest,0.764060
SVM,0.774886


In [32]:
# 1. Optimization History
plot_optimization_history(study).show()

In [33]:
# 3. Slice Plot
plot_slice(study).show()

In [34]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [35]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 2.9 MB/s eta 0:00:00


In [36]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-11-25 19:55:54,206] A new study created in memory with name: no-name-ed99dbe5-8128-45b1-83aa-fa9997b69494


[0]	train-mlogloss:0.80894	eval-mlogloss:0.78145
[1]	train-mlogloss:0.62087	eval-mlogloss:0.57816
[2]	train-mlogloss:0.49571	eval-mlogloss:0.45005
[3]	train-mlogloss:0.41192	eval-mlogloss:0.35518
[4]	train-mlogloss:0.36167	eval-mlogloss:0.30326
[5]	train-mlogloss:0.31922	eval-mlogloss:0.25854
[6]	train-mlogloss:0.28401	eval-mlogloss:0.22145
[7]	train-mlogloss:0.28400	eval-mlogloss:0.22150
[8]	train-mlogloss:0.26923	eval-mlogloss:0.20239
[9]	train-mlogloss:0.26727	eval-mlogloss:0.19775
[10]	train-mlogloss:0.26670	eval-mlogloss:0.19789
[11]	train-mlogloss:0.26681	eval-mlogloss:0.19831
[12]	train-mlogloss:0.26666	eval-mlogloss:0.19850
[13]	train-mlogloss:0.26601	eval-mlogloss:0.19821
[14]	train-mlogloss:0.26570	eval-mlogloss:0.19884
[15]	train-mlogloss:0.26441	eval-mlogloss:0.19776
[16]	train-mlogloss:0.26438	eval-mlogloss:0.19784
[17]	train-mlogloss:0.26436	eval-mlogloss:0.19793
[18]	train-mlogloss:0.26456	eval-mlogloss:0.19757
[19]	train-mlogloss:0.26362	eval-mlogloss:0.19533
[20]	train

[I 2025-11-25 19:55:54,505] Trial 0 finished with value: 1.0 and parameters: {'lambda': 5.79529897579039e-07, 'alpha': 0.010107584597789822, 'eta': 0.24718126514870087, 'gamma': 3.9314093384328194e-05, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.6281728005677456, 'colsample_bytree': 0.96713230664329}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.89047	eval-mlogloss:0.87692
[1]	train-mlogloss:0.76681	eval-mlogloss:0.73831
[2]	train-mlogloss:0.64243	eval-mlogloss:0.60548
[3]	train-mlogloss:0.53552	eval-mlogloss:0.49428
[4]	train-mlogloss:0.46428	eval-mlogloss:0.42318
[5]	train-mlogloss:0.40308	eval-mlogloss:0.35388
[6]	train-mlogloss:0.36455	eval-mlogloss:0.31439
[7]	train-mlogloss:0.32552	eval-mlogloss:0.27837
[8]	train-mlogloss:0.29794	eval-mlogloss:0.24743
[9]	train-mlogloss:0.27426	eval-mlogloss:0.22056
[10]	train-mlogloss:0.26720	eval-mlogloss:0.21159
[11]	train-mlogloss:0.26056	eval-mlogloss:0.20368
[12]	train-mlogloss:0.24900	eval-mlogloss:0.19270
[13]	train-mlogloss:0.24942	eval-mlogloss:0.19432
[14]	train-mlogloss:0.24890	eval-mlogloss:0.19283
[15]	train-mlogloss:0.24758	eval-mlogloss:0.19093
[16]	train-mlogloss:0.24613	eval-mlogloss:0.18891
[17]	train-mlogloss:0.24574	eval-mlogloss:0.18981
[18]	train-mlogloss:0.23803	eval-mlogloss:0.18118
[19]	train-mlogloss:0.23735	eval-mlogloss:0.18074
[20]	train

[I 2025-11-25 19:55:55,043] Trial 1 finished with value: 1.0 and parameters: {'lambda': 5.568164279565003e-07, 'alpha': 2.0876369263302372e-07, 'eta': 0.17528269568198912, 'gamma': 0.0003522807854680442, 'max_depth': 6, 'min_child_weight': 6, 'subsample': 0.47823309153049415, 'colsample_bytree': 0.7689923898471073}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.95630	eval-mlogloss:0.95041


[I 2025-11-25 19:56:14,292] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:0.80684	eval-mlogloss:0.78676


[I 2025-11-25 19:56:14,339] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 5.79529897579039e-07, 'alpha': 0.010107584597789822, 'eta': 0.24718126514870087, 'gamma': 3.9314093384328194e-05, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.6281728005677456, 'colsample_bytree': 0.96713230664329}
Best accuracy: 1.0


In [37]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()
